In [70]:
# lets load the data from train.csv and test.csv files
import pandas as pd
import numpy as np

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


C:\Users\kb\AppData\Local\Temp\ipykernel_34644\3475752379.py:5: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('train.csv')


In [39]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype   
---  ------                    --------------   -----   
 0   ID                        100000 non-null  object  
 1   Customer_ID               100000 non-null  object  
 2   Month                     100000 non-null  object  
 3   Name                      90015 non-null   object  
 4   Age                       712 non-null     category
 5   SSN                       100000 non-null  object  
 6   Occupation                100000 non-null  object  
 7   Annual_Income             75184 non-null   category
 8   Monthly_Inhand_Salary     99336 non-null   float64 
 9   Num_Bank_Accounts         100000 non-null  float64 
 10  Num_Credit_Card           100000 non-null  float64 
 11  Interest_Rate             100000 non-null  float64 
 12  Num_of_Loan               100000 non-null  object  
 13  Type_of_Loan              8859

In [71]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd

class NumericCleaner(BaseEstimator, TransformerMixin):
   def __init__(self):
      self.columns_to_clean =  ['Age', 'Annual_Income', 'Num_of_Loan', 'Num_of_Delayed_Payment',
       'Changed_Credit_Limit', 'Outstanding_Debt', 'Amount_invested_monthly', 'Monthly_Balance']

   def fit(self, X, y=None):
      # No fitting necessary for this transformer
      return self

   def transform(self, X):
      # Make a copy of the DataFrame to avoid modifying the original data
      X = X.copy()
            
      # Apply cleaning function to each specified column
      for column in self.columns_to_clean:
         X[column] = pd.to_numeric(X[column].str.replace(r'[^0-9.-]', '', regex=True), errors='coerce')

        
      return X

In [72]:

# Create an instance of the transformer
numeric_cleaner = NumericCleaner()

# Assuming 'train' is your DataFrame
train_cleaned = numeric_cleaner.fit_transform(train)

# Check the result
print(train_cleaned[columns_to_clean].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Age                      100000 non-null  int64  
 1   Annual_Income            100000 non-null  float64
 2   Num_of_Loan              100000 non-null  int64  
 3   Num_of_Delayed_Payment   92998 non-null   float64
 4   Changed_Credit_Limit     97909 non-null   float64
 5   Outstanding_Debt         100000 non-null  float64
 6   Amount_invested_monthly  95521 non-null   float64
 7   Monthly_Balance          97132 non-null   float64
dtypes: float64(6), int64(2)
memory usage: 6.1 MB
None


In [67]:
train.describe()

,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Delay_from_due_date,Num_Credit_Inquiries,Credit_Utilization_Ratio,Total_EMI_per_month
count,84998.000000,100000.000000,100000.00000,100000.000000,100000.000000,98035.000000,100000.000000,100000.000000
mean,4194.170850,17.091280,22.47443,72.466040,21.068780,27.754251,32.285173,1403.118217
std,3183.686167,117.404834,129.05741,466.422621,14.860104,193.177339,5.116875,8306.041270
min,303.645417,-1.000000,0.00000,1.000000,-5.000000,0.000000,20.000000,0.000000
25%,1625.568229,3.000000,4.00000,8.000000,10.000000,3.000000,28.052567,30.306660
50%,3093.745000,6.000000,5.00000,13.000000,18.000000,6.000000,32.305784,69.249473
75%,5957.448333,7.000000,7.00000,20.000000,28.000000,9.000000,36.496663,161.224249
max,15204.633333,1798.000000,1499.00000,5797.000000,67.000000,2597.000000,50.000000,82331.000000


In [73]:
hi_lo_bounds = {
    # Numerical Columns
    'Age': {'lo': 0, 'hi': 100},
    'Annual_Income': {'lo': 5000, 'hi': 500000},
    'Monthly_Inhand_Salary': {'lo': 500, 'hi': 30000},
    'Num_Bank_Accounts': {'lo': 0, 'hi': 20},
    'Num_Credit_Card': {'lo': 0, 'hi': 15},
    'Interest_Rate': {'lo': 0, 'hi': 50},
    'Delay_from_due_date': {'lo': 0, 'hi': 60},
    'Num_Credit_Inquiries': {'lo': 0, 'hi': 30},
    'Changed_Credit_Limit': {'lo': 0, 'hi': 30},  # Bounds after converting to numeric
    
    # Categorical Columns (ordered)
    'Credit_Mix': {'lo': 0, 'hi': 2},  # Assuming "Bad" = 0, "Standard" = 1, "Good" = 2

    # Categorical Column (nominal)
    'Occupation': {'lo': 0, 'hi': 14}  # No bounds, only impute missing values
}


In [25]:
train['Credit_Mix'].value_counts()
#change the credit mix to categorical
train['Credit_Mix'] = train['Credit_Mix'].astype('category')
train['Credit_Mix'].value_counts()
#x = train['Credit_Mix'].cat.codes
#x.value_counts()


Credit_Mix
Standard    36479
Good        24337
_           20195
Bad         18989
Name: count, dtype: int64

In [26]:
x = train['Credit_Mix'].cat.codes
x.value_counts()

2    36479
1    24337
3    20195
0    18989
Name: count, dtype: int64

In [55]:
import pandas as pd

# Load your data
train = pd.read_csv('train.csv')

# List of columns to clean
columns_to_clean = [
    'Age', 'Annual_Income', 'Num_of_Loan', 'Num_of_Delayed_Payment',
    'Changed_Credit_Limit', 'Outstanding_Debt', 'Amount_invested_monthly', 'Monthly_Balance'
]

# Function to clean and convert columns
def clean_and_convert(column):
    # Replace non-numeric characters and convert to numeric
    train[column] = pd.to_numeric(train[column].str.replace(r'[^0-9.-]', '', regex=True), errors='coerce')
    print(column)
    print(train[column].isnull().sum())

# Apply cleaning function to each column
for column in columns_to_clean:
    clean_and_convert(column)

# Check the result
print(train[columns_to_clean].info())

C:\Users\kb\AppData\Local\Temp\ipykernel_34644\343377441.py:4: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('train.csv')


Age
0
Annual_Income
0
Num_of_Loan
0
Num_of_Delayed_Payment
7002
Changed_Credit_Limit
2091
Outstanding_Debt
0
Amount_invested_monthly
4479
Monthly_Balance
2868
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Age                      100000 non-null  int64  
 1   Annual_Income            100000 non-null  float64
 2   Num_of_Loan              100000 non-null  int64  
 3   Num_of_Delayed_Payment   92998 non-null   float64
 4   Changed_Credit_Limit     97909 non-null   float64
 5   Outstanding_Debt         100000 non-null  float64
 6   Amount_invested_monthly  95521 non-null   float64
 7   Monthly_Balance          97132 non-null   float64
dtypes: float64(6), int64(2)
memory usage: 6.1 MB
None


In [2]:
numeric_attributes = ['Age', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate',  'Num_of_Loan', 'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Outstanding_Debt', 'Monthly_Balance', 'AmountInvested']

for attribute in numeric_attributes:
      train[attribute] = pd.to_numeric(train[attribute], errors='coerce')
      print(attribute)
      print(train[attribute].isnull().sum())

Age
4939
Annual_Income
6980
Monthly_Inhand_Salary
15002
Num_Bank_Accounts
0
Num_Credit_Card
0
Interest_Rate
0
Num_of_Loan
4785
Delay_from_due_date
0
Num_of_Delayed_Payment
9746
Changed_Credit_Limit
2091
Outstanding_Debt
1009
Monthly_Balance
1209


In [101]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.utils.validation import check_is_fitted
import pandas as pd

class CreditHistoryTransformer(BaseEstimator, TransformerMixin):
   def __init__(self):
      self.column = 'Credit_History_Age'
        

   def fit(self, X, y=None): 
      assert isinstance(X, pd.DataFrame) and self.column in X.columns
      #set number of features
      self._n_features = X.shape[1]
      return self
   
   def transform(self, X):
      check_is_fitted(self, '_n_features')
      assert isinstance(X, pd.DataFrame) and self.column in X.columns   
      #now lets transformt the data 
      # we need to convert the verbal description to numerical values
      # it's in this form '22 Years and 1 Months'
      # first we change the column to string type 
      #tthen process the string to get the numerical values
      # then multiply the years by 12 and add the months
      X[self.column] = X[self.column].astype(str)

      years = X[self.column].str.extract(r'(\d+)\s*Years?')[0].fillna(0).astype(int)
      months = X[self.column].str.extract(r'(\d+)\s*Months?')[0].fillna(0).astype(int)

    # Calculate total months
      X[self.column] = years * 12 + months
      return X

      

   
     


In [102]:
creditHistory = CreditHistoryTransformer()
creditHistory.fit(train)
x=creditHistory.transform(train)
x.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,265,No,49.574949,80.4152954390025,High_spent_Small_value_payments,312.494088679437,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,0,No,49.574949,118.280221622367,Low_spent_Large_value_payments,284.629162496072,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,267,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.209862853791,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,268,No,49.574949,199.458074391071,Low_spent_Small_value_payments,223.451309727368,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,269,No,49.574949,41.4201530862173,High_spent_Medium_value_payments,341.489231032222,Good


In [76]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.utils.validation import check_is_fitted
import pandas as pd

class PaymentBehaviourTransformer(BaseEstimator, TransformerMixin):
   def __init__(self):
      self.charge_mapping = {'Low_spent': 0, 'Medium_spent': 1, 'High_spent': 2}
      self.payment_mapping = {'Small_value_payments': 0, 'Medium_value_payments': 1, 'Large_value_payments': 2}
      self.columns = 'Payment_Behaviour'

   def fit(self, X, y=None):
      assert isinstance(X, pd.DataFrame)
      'Payment_Behaviour' in X.columns

      return self
   
   def transform(self, X):
      charge, payment = X['Payment_Behaviour'].apply(self.split_payment_behavior)
      X['Charge'] = charge
      X['Payment'] = payment
      X.drop('Payment_Behaviour', axis=1, inplace=True)
      return X
   
   def split_payment_behavior(self, value):
      if isinstance(value, str):
         # Split the value into components based on the underscore
         parts = value.split('_')
         
         # The first part corresponds to charge, and the last two parts correspond to payment
         charge_part = '_'.join(parts[:2])  # Join the first two parts for charge
         payment_part = '_'.join(parts[2:])  # Join the rest for payment
         
         # Get the corresponding numerical values from the mappings
         charge_value = self.charge_mapping.get(charge_part, np.nan)
         payment_value = self.payment_mapping.get(payment_part, np.nan)
         
         return pd.Series([charge_value, payment_value])
      else:
         return pd.Series([np.nan, np.nan])  # Return NaN for non-string values

   # Assuming X is your DataFrame


           

In [73]:
PaymentBehaviourTransformer  = PaymentBehaviourTransformer()
PaymentBehaviourTransformer.fit(train)
x = PaymentBehaviourTransformer.transform(train)

In [74]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 29 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

In [33]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.utils.validation import check_is_fitted
import pandas as pd

class LoanType(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.ml_binarizer = MultiLabelBinarizer()

    def fit(self, X, y=None):
        assert isinstance(X, pd.DataFrame) and 'Type_of_Loan' in X.columns
        # Ensure the column is treated as a string and handle NaN values
        loan_types = X['Type_of_Loan'].fillna('Not Specified').astype(str).str.split(', ')
        # Clean up the loan types to remove unwanted phrases
        loan_types = loan_types.apply(lambda x: [item.strip() for item in x if 'and' not in item])
        self.ml_binarizer.fit(loan_types)      
        self._n_features = len(self.ml_binarizer.classes_)
        return self 

    def transform(self, X):
        # Check if the transformer has been fitted
        check_is_fitted(self, '_n_features')
        assert isinstance(X, pd.DataFrame) and 'Type_of_Loan' in X.columns
        # Ensure the column is treated as a string and handle NaN values
        loan_types = X['Type_of_Loan'].fillna('Not Specified').astype(str).str.split(', ')
        # Clean up the loan types to remove unwanted phrases
        loan_types = loan_types.apply(lambda x: [item.strip() for item in x if 'and' not in item])
        loan_type_dummies = self.ml_binarizer.transform(loan_types)
        
        # Create a DataFrame for the binary columns
        loan_type_df = pd.DataFrame(loan_type_dummies, columns=self.ml_binarizer.classes_, index=X.index)
        
        # Concatenate the new binary columns with the original DataFrame
        X = pd.concat([X, loan_type_df], axis=1)
        return X



In [15]:
train['Type_of_Loan'].info()



<class 'pandas.core.series.Series'>
RangeIndex: 100000 entries, 0 to 99999
Series name: Type_of_Loan
Non-Null Count  Dtype 
--------------  ----- 
88592 non-null  object
dtypes: object(1)
memory usage: 781.4+ KB


In [34]:
Loan = LoanType()
Loan.fit(train)
x=Loan.transform(train)
x.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Score,Auto Loan,Credit-Builder Loan,Debt Consolidation Loan,Home Equity Loan,Mortgage Loan,Not Specified,Payday Loan,Personal Loan,Student Loan
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,1,1,0,0,0,0,0,1,0
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,1,1,0,0,0,0,0,1,0
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,1,1,0,0,0,0,0,1,0
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,1,1,0,0,0,0,0,1,0
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,1,1,0,0,0,0,0,1,0


In [68]:
x.columns

Index(['ID', 'Customer_ID', 'Month', 'Name', 'Age', 'SSN', 'Occupation',
       'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
       'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan',
       'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt',
       'Credit_Utilization_Ratio', 'Credit_History_Age',
       'Payment_of_Min_Amount', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Monthly_Balance', 'Credit_Score', 'Charge',
       'Payment'],
      dtype='object')

In [70]:
x['Charge'].isnull().sum()

0

In [69]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 29 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

In [37]:
from numpy import NAN
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted

class HiLoImputer(BaseEstimator, TransformerMixin):
    def __init__(self, lo, hi, columnName):
        self.hi = hi
        self.lo = lo
        self.columnName = columnName
        self.isCategory = False

    def fit(self, X, y=None):
        assert isinstance(X, pd.DataFrame) and len(X.columns) > 1
        assert self.columnName in X.columns
        
        # Check if the column is categorical
        self.isCategory = pd.api.types.is_categorical_dtype(X[self.columnName]) or X[self.columnName].dtype.name == 'object'
        if self.isCategory:
            #convert to category

            self.categories_ = X[self.columnName].astype('category').cat.categories

        # Store the number of features
        self._n_features = X.shape[1]
        return self

    def transform(self, X):
        check_is_fitted(self, '_n_features')
        assert isinstance(X, pd.DataFrame) and X.shape[1] == self._n_features

        if self.isCategory:
            # Convert categories to codes for hi/lo processing
            X[self.columnName] = X[self.columnName].astype('category').cat.codes

            # Apply hi/lo bounds, setting out-of-bounds to NaN
            X[self.columnName] = np.where(
                (X[self.columnName] > self.hi) | (X[self.columnName] < self.lo), 
                np.nan, X[self.columnName]
            )
            
            # Replace NaN values with the mode per Customer_ID group
            X[self.columnName] = X.groupby('Customer_ID', group_keys=False)[self.columnName].apply(
                lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else x)
            )

            # Convert back to categories
            X[self.columnName] = pd.Categorical.from_codes(
                X[self.columnName].fillna(-1).astype(int), categories=self.categories_, ordered=True
            ).remove_unused_categories()

        else:
            # Apply hi/lo bounds for numerical data
            X[self.columnName] = np.where(
                (X[self.columnName] > self.hi) | (X[self.columnName] < self.lo), 
                np.nan, X[self.columnName]
            )

            # Replace NaN values with the mode per Customer_ID group
            X[self.columnName] = X.groupby('Customer_ID', group_keys=False)[self.columnName].apply(
                lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else x)
            )

        return X


In [125]:
train['Occupation'].value_counts()

Occupation
Lawyer           7096
Engineer         6864
Architect        6824
Mechanic         6776
Accountant       6744
Scientist        6744
Developer        6720
Media_Manager    6720
Teacher          6672
Entrepreneur     6648
Doctor           6568
Journalist       6536
Manager          6432
Musician         6352
Writer           6304
Name: count, dtype: int64

In [124]:
#change occupation to categorical data
train['Occupation'] = train['Occupation'].astype('category')
imputer = HiLoImputer(0,14, 'Occupation')
imputer.fit_transform(train)
train.head()

C:\Users\kb\AppData\Local\Temp\ipykernel_2156\3718962601.py:19: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  self.isCategory = pd.api.types.is_categorical_dtype(X[self.columnName])


,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.4152954390025,High_spent_Small_value_payments,312.494088679437,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.280221622367,Low_spent_Large_value_payments,284.629162496072,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.209862853791,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.458074391071,Low_spent_Small_value_payments,223.451309727368,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.4201530862173,High_spent_Medium_value_payments,341.489231032222,Good


In [74]:
for column, bounds in hi_lo_bounds.items():
    lo = bounds['lo']
    hi = bounds['hi']
    
    # Apply HiLoImputer only for imputation if bounds are None
    if lo is None and hi is None:
        #HiLoImputer(0,14, 'Occupation')
        hi_lo_imputer = HiLoImputer(lo=float('-inf'), hi=float('inf'), columnName=column)
    else:
        hi_lo_imputer = HiLoImputer(lo=lo, hi=hi, columnName=column)
    data = train
    print(column)
    train = hi_lo_imputer.fit_transform(train)
    print(train[column].isnull().sum())   


Age


C:\Users\kb\AppData\Local\Temp\ipykernel_34644\2918858931.py:19: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  self.isCategory = pd.api.types.is_categorical_dtype(X[self.columnName]) or X[self.columnName].dtype.name == 'object'


92304
Annual_Income


C:\Users\kb\AppData\Local\Temp\ipykernel_34644\2918858931.py:19: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  self.isCategory = pd.api.types.is_categorical_dtype(X[self.columnName]) or X[self.columnName].dtype.name == 'object'


24816
Monthly_Inhand_Salary


...

98936
Credit_Mix


C:\Users\kb\AppData\Local\Temp\ipykernel_34644\2918858931.py:19: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  self.isCategory = pd.api.types.is_categorical_dtype(X[self.columnName]) or X[self.columnName].dtype.name == 'object'


0
Occupation


C:\Users\kb\AppData\Local\Temp\ipykernel_34644\2918858931.py:19: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  self.isCategory = pd.api.types.is_categorical_dtype(X[self.columnName]) or X[self.columnName].dtype.name == 'object'


0


In [62]:
train['Occupation'].value_counts()

Occupation
_______          7062
Lawyer           6575
Architect        6355
Engineer         6350
Scientist        6299
Mechanic         6291
Accountant       6271
Developer        6235
Media_Manager    6232
Teacher          6215
Entrepreneur     6174
Doctor           6087
Journalist       6085
Manager          5973
Musician         5911
Writer           5885
Name: count, dtype: int64

In [61]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

In [41]:
# so the data has some realy high unrealeastic values in age, num_of_bank_accounts, num_of_credit_card, num_of_loan
# lets get a feel and get a right range for these values
# age hi value is 100, low value is 0
# num_of_bank_accounts hi value is 10, low value is 1
# num_of_credit_card hi value is 10, low value is 1
# num_of_loan hi value is 10, low value is 1
# annual income lo is 0
# monthly inhand salary lo is 0


# so now lets create a transformer



Age
38.0      2833
28.0      2829
31.0      2806
26.0      2792
32.0      2749
          ... 
5741.0       1
7178.0       1
5621.0       1
1908.0       1
1342.0       1
Name: count, Length: 1661, dtype: int64
Num_Bank_Accounts
6       13001
7       12823
8       12765
4       12186
5       12118
        ...  
1626        1
1470        1
887         1
211         1
697         1
Name: count, Length: 943, dtype: int64
Num_Credit_Card
5       18459
7       16615
6       16559
4       14030
3       13277
        ...  
791         1
1118        1
657         1
640         1
679         1
Name: count, Length: 1179, dtype: int64
Num_of_Loan
3.0       14386
2.0       14250
4.0       14016
0.0       10380
1.0       10083
          ...  
1444.0        1
392.0         1
841.0         1
1015.0        1
966.0         1
Name: count, Length: 399, dtype: int64
110.93450521244253
17.09128
22.47443
2.780339232263824
33.0
6.0
5.0
3.0
38.0
6
5
3.0
